In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('processed_data/with_ranges_features.csv')

# Determine the number of classes and create a global label encoder


2024-06-22 12:38:21.656849: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-22 12:38:21.688547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-22 12:38:21.689024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 12:38:22.229809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Extract features and label
df = data[['Elevation', 'Distance', 'HeartRateQuotient', 'HeartRateRange', 'Cadence', 'Speed', 'SpeedRange', 'SpeedQuotient', 'HeartRateClass']]
df.dropna()

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
    
features = df[['Elevation', 'Distance', 'HeartRateQuotient', 'HeartRateRange', 'Cadence', 'Speed', 'SpeedRange', 'SpeedQuotient']].to_numpy()
features = features.reshape(features.shape[0], 1, features.shape[1])

labels = df['HeartRateClass'].to_numpy()
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['HeartRateClass'])

num_classes = len(label_encoder.classes_)

#labels = to_categorical(label, num_classes=num_classes)
    
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [3]:
from tcn import TCN, compiled_tcn

def build_model(middle_layer):
    model = keras.Sequential([
        keras.layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        middle_layer,
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.summary()
    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['acc'])
    return model

In [4]:
def get_tcn_layer():
    return TCN(
        nb_filters=30,
        kernel_size=3,
        dilations=[2**i for i in range(6)],
    )
tcn_model = build_model(get_tcn_layer())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 30)                31050     
                                                                 
 dense (Dense)               (None, 10)                310       
                                                                 
Total params: 31,360
Trainable params: 31,360
Non-trainable params: 0
_________________________________________________________________


In [5]:
def get_lstm_layer():
    return keras.layers.LSTM(100)

lstm_model = build_model(get_lstm_layer())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               43600     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 44,610
Trainable params: 44,610
Non-trainable params: 0
_________________________________________________________________


2024-06-22 12:38:44.570443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-22 12:38:44.571440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-22 12:38:44.572148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [6]:
lstm_history = lstm_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=256)

Epoch 1/3


2024-06-22 12:38:48.541274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-22 12:38:48.542182: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-22 12:38:48.543049: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5168/5174 [============================>.] - ETA: 0s - loss: nan - acc: 4.2328e-05

2024-06-22 12:38:59.472671: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-22 12:38:59.473943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-22 12:38:59.474720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5174/5174 [==============================] - 12s 2ms/step - loss: nan - acc: 4.2282e-05 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
5174/5174 [==============================] - 11s 2ms/step - loss: nan - acc: 2.2651e-06 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/3
5174/5174 [==============================] - 11s 2ms/step - loss: nan - acc: 2.2651e-06 - val_loss: nan - val_acc: 0.0000e+00


In [7]:
tcn_history = tcn_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=256)

Epoch 1/3
5174/5174 [==============================] - 90s 17ms/step - loss: nan - acc: 9.8155e-06 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
5174/5174 [==============================] - 88s 17ms/step - loss: nan - acc: 2.2651e-06 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/3
5174/5174 [==============================] - 87s 17ms/step - loss: nan - acc: 2.2651e-06 - val_loss: nan - val_acc: 0.0000e+00
